# Homework 2: Semi-structured Data

By Gary Chen and Mohamad Nayal

Initial setup for getting Youtube Data Api was done with the help of code from the following site:

https://www.thepythoncode.com/article/using-youtube-api-in-python#Extracting_YouTube_Comments

In [1]:
import os
os.chdir("C:/Users/otan")

In [9]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import re
import os
import pickle

SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [10]:
#Youtube data api authentication script provided by 
#https://www.thepythoncode.com/article/using-youtube-api-in-python#Extracting_YouTube_Comments

#defining youtube authentication script for first time use
def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "credentials.json"
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return build(api_service_name, api_version, credentials=creds)

# authenticate to YouTube API
youtube = youtube_authenticate()

In [11]:
def get_video_id_by_url(url):
    """
    Return the Video ID from the video `url`
    """
    # split URL parts
    parsed_url = p.urlparse(url)
    # get the video ID by parsing the query of the URL
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")

In [12]:
def get_video_details(youtube, **kwargs):
    return youtube.videos().list(
        part="snippet,contentDetails,statistics",
        **kwargs
    ).execute()

In [13]:
def get_comments(youtube, **kwargs):
    commentsList=youtube.commentThreads().list(
        part="snippet",
        **kwargs
    ).execute()
    return commentsList

In [14]:
# URL can be a channel or a video, to extract comments
url = "https://www.youtube.com/watch?v=BTVMLRSsb3o"
if "watch" in url:
    # that's a video
    video_id = get_video_id_by_url(url)
    params = {
        'videoId': video_id, 
        'maxResults': 100,
        'order': 'relevance', # default is 'time' (newest)
    }
else:
    # should be a channel
    channel_id = get_channel_id_by_url(url)
    params = {
        'allThreadsRelatedToChannelId': channel_id, 
        'maxResults': 100,
        'order': 'relevance', # default is 'time' (newest)
    }
# get the first 2 pages (2 API requests)
n_pages = 4
for i in range(n_pages):
    # make API call to get all comments from the channel (including posts & videos)
    response = get_comments(youtube, **params)
    items = response.get("items")
    # if items is empty, breakout of the loop
    if not items:
        break
    for item in items:
        comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        updated_at = item["snippet"]["topLevelComment"]["snippet"]["updatedAt"]
        like_count = item["snippet"]["topLevelComment"]["snippet"]["likeCount"]
        comment_id = item["snippet"]["topLevelComment"]["id"]
        print(f"""\
        Comment: {comment}
        Likes: {like_count}
        Updated At: {updated_at}
        ==================================\
        """)
    if "nextPageToken" in response:
        # if there is a next page
        # add next page token to the params we pass to the function
        params["pageToken"] =  response["nextPageToken"]
        response2= get_comments(youtube, **params)
        items2 = response2.get("items")
        if "nextPageToken" in response:
          # if there is a next page
        # add next page token to the params we pass to the function
            params["pageToken"] =  response["nextPageToken"]
            response3= get_comments(youtube, **params)
            items3 = response3.get("items")
            if "nextPageToken" in response:
          # if there is a next page
        # add next page token to the params we pass to the function
                params["pageToken"] =  response["nextPageToken"]
                response4= get_comments(youtube, **params)
                items4 = response4.get("items")
    else:
        # must be end of comments!!!!
        break
    print("*"*70)

        Comment: Thanks for watching guys!! 👊🏼🙇🏻‍♂️<br>Next Monday ⚠️ Overtime 29 🚨 <br><br>Download Mech Arena for FREE here (<a href="https://clik.cc/4eqWG)">https://clik.cc/4eqWG)</a> and get our special starter pack to help kickstart your game! ⭐️
        Likes: 3109
        Updated At: 2021-08-23T22:40:47Z
        Comment: Dude perfect should open up an amusement park! I’d Go!!!
        Likes: 6410
        Updated At: 2021-08-23T22:06:52Z
        Comment: their production value is insane
        Likes: 507
        Updated At: 2021-08-25T09:32:09Z
        Comment: Man it’s been amazing watch DP grow and evolve over these years. This is amazing
        Likes: 210
        Updated At: 2021-08-25T18:27:45Z
        Comment: Yet another epic race! Also, need an underwater scooter ASAP 👀
        Likes: 244
        Updated At: 2021-08-26T00:41:16Z
        Comment: “Went Ahead and shaved off all my body hair this morning” says  Ty with a 6 inch beard
        Likes: 257
        Updated At: 2

**********************************************************************
        Comment: I’d like to personally give an award to the people who commented “first” to commemorate their stupendous achievement
        Likes: 183
        Updated At: 2021-08-23T22:04:04Z
        Comment: Coby almost won! By one point! Man!
        Likes: 1
        Updated At: 2021-08-27T03:51:00Z
        Comment: I like how they put needles on the submarine so the mines would pop
        Likes: 1
        Updated At: 2021-08-24T16:24:13Z
        Comment: I feel like I just watch the battles to find out if Coby wins or not.
        Likes: 133
        Updated At: 2021-08-24T05:55:52Z
        Comment: I just want poor Coby to win again. He’s so close yet so far away
        Likes: 0
        Updated At: 2021-08-24T16:06:54Z
        Comment: Bruh he actually blew it 💀
        Likes: 0
        Updated At: 2021-08-28T11:17:01Z
        Comment: I wish I could fish like that😂
        Likes: 0
        Updated At: 2021-0

**********************************************************************
        Comment: I agree mustard is nasty
        Likes: 2
        Updated At: 2021-08-28T13:28:25Z
        Comment: No fish were harmed during the making if this video.
        Likes: 2
        Updated At: 2021-08-26T09:00:47Z
        Comment: Such a cool video!  Awesome
        Likes: 2
        Updated At: 2021-08-26T03:29:56Z
        Comment: Tell the dudes all 56.6 million of the subscribers said to give you a raise if they still haven&#39;t
        Likes: 2
        Updated At: 2021-08-24T13:45:19Z
        Comment: AWESOME VID !!!!!!!!!!!!!!!!!!!!!!!!!!
        Likes: 2
        Updated At: 2021-08-24T12:31:45Z
        Comment: that hot dog took 36 minutes of their lives
        Likes: 1
        Updated At: 2021-08-28T00:11:17Z
        Comment: Do a top 10 superheros
        Likes: 1
        Updated At: 2021-08-27T23:29:16Z
        Comment: missing trick shots
        Likes: 1
        Updated At: 2021-08-27T06:01

**********************************************************************
        Comment: Nobody: <br>Coby: I’ve never had a hotdog at 8am before
        Likes: 0
        Updated At: 2021-08-24T16:07:45Z
        Comment: I love how Cory is wearing yellow and he hates mustard.
        Likes: 26
        Updated At: 2021-08-23T22:33:57Z
        Comment: Boi ur the channel that make my day better, thank you for the past six year:D
        Likes: 0
        Updated At: 2021-08-26T01:00:03Z
        Comment: Who realized that they used Ty&#39;s OT 28 Cool Not Cool item in round 1?
        Likes: 0
        Updated At: 2021-08-25T23:24:07Z
        Comment: Why is mech arena exactly like war robots but less detailed either way loved the vid
        Likes: 0
        Updated At: 2021-08-28T03:00:46Z
        Comment: The savory church serendipitously choke because oak consecutively grin anenst a goofy sailor. fantastic, panicky polo
        Likes: 0
        Updated At: 2021-08-26T22:34:23Z
        Com

**********************************************************************


In [15]:
def getList(dict):
    return dict.keys()
      

print(getList(response))

response['pageInfo']

dict_keys(['kind', 'etag', 'nextPageToken', 'pageInfo', 'items'])


{'totalResults': 100, 'resultsPerPage': 100}

In [16]:
print(len(items))
print(len(items2))
print(len(items3))
print(len(items4))

#

import pandas as pd
import json



#creating one dictionary of comments info from four dictionary opbjects
listOfItemObjects=[items,items2,items3, items4]

#items var is a list/dict
#use json.dumps to format into JSON

json_object1=json.dumps(items, indent = 2)
json_object2=json.dumps(items2, indent = 2)
json_object3=json.dumps(items3, indent = 2)
json_object4=json.dumps(items4, indent = 2)


100
100
100
100


Questions: 

-Does time of comment post correlate with likes? hr of day 8-4 working hours, 4-12 night, 12-8 late night

-Does time between video post and comment post correlate with likes?

-Does the length of comments have an impact on likes?

-Does comments with content creator names result in higher likes?

In [17]:
#turn JSON into 4 different Pandas DF
js_items1 = json.loads(json_object1)
js_items2 = json.loads(json_object2)
js_items3 = json.loads(json_object3)
js_items4 = json.loads(json_object4)
df_items1 = pd.json_normalize(js_items1)
df_items2 = pd.json_normalize(js_items2)
df_items3 = pd.json_normalize(js_items3)
df_items4 = pd.json_normalize(js_items4)
#appending all dataframes together to make one datafram
#The 4 DF are being stacked
df_items1_2=df_items1.append(df_items2, ignore_index=True)
df_items1_2_3=df_items1_2.append(df_items3, ignore_index=True)
df_items1_2_3_4=df_items1_2_3.append(df_items4, ignore_index=True)
df_items=df_items1_2_3_4
#print(type(js_items))
#print(type(df_items))

#display(df_items.head())

In [18]:
print(df_items['snippet.topLevelComment.snippet.textOriginal'].head())

0    Nobody: \nCoby: I’ve never had a hotdog at 8am...
1    I love how Cory is wearing yellow and he hates...
2    Boi ur the channel that make my day better, th...
3    Who realized that they used Ty's OT 28 Cool No...
4    Why is mech arena exactly like war robots but ...
Name: snippet.topLevelComment.snippet.textOriginal, dtype: object


In [19]:
#trim DF into the columns we need

df_items_trim = df_items[['id',
 
'snippet.topLevelComment.snippet.textDisplay',

'snippet.topLevelComment.snippet.textOriginal',

'snippet.topLevelComment.snippet.likeCount', 

'snippet.topLevelComment.snippet.publishedAt',

'snippet.topLevelComment.snippet.updatedAt',

 'snippet.totalReplyCount']] 

In [20]:
#rename bulky column names
df_comment_stats = df_items_trim.rename(columns = {'id': 'id',
 
'snippet.topLevelComment.snippet.textDisplay' : 'comment_display',

'snippet.topLevelComment.snippet.textOriginal': 'comment_text',

'snippet.topLevelComment.snippet.likeCount': 'like_count', 

'snippet.topLevelComment.snippet.publishedAt': 'comment_publish_date',

'snippet.topLevelComment.snippet.updatedAt':'comment_up_date',

 'snippet.totalReplyCount': 'reply_count'})

display(df_items_trim.head())

display(df_comment_stats.head())

,id,snippet.topLevelComment.snippet.textDisplay,snippet.topLevelComment.snippet.textOriginal,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.totalReplyCount
0,UgxNaXAx1pl7S2dWGiN4AaABAg,Nobody: <br>Coby: I’ve never had a hotdog at 8...,Nobody: \nCoby: I’ve never had a hotdog at 8am...,0,2021-08-24T16:07:45Z,2021-08-24T16:07:45Z,0
1,Ugx0awVaxhL-rBNKrNx4AaABAg,I love how Cory is wearing yellow and he hates...,I love how Cory is wearing yellow and he hates...,26,2021-08-23T22:33:57Z,2021-08-23T22:33:57Z,0
2,UgxFSWHcREJ0oP_SR8p4AaABAg,"Boi ur the channel that make my day better, th...","Boi ur the channel that make my day better, th...",0,2021-08-26T01:00:03Z,2021-08-26T01:00:03Z,0
3,UgzpQ5tGyMgiVxxRbGx4AaABAg,Who realized that they used Ty&#39;s OT 28 Coo...,Who realized that they used Ty's OT 28 Cool No...,0,2021-08-25T23:24:07Z,2021-08-25T23:24:07Z,0
4,UgynfhIedxMBipSuj1p4AaABAg,Why is mech arena exactly like war robots but ...,Why is mech arena exactly like war robots but ...,0,2021-08-28T03:00:46Z,2021-08-28T03:00:46Z,0


,id,comment_display,comment_text,like_count,comment_publish_date,comment_up_date,reply_count
0,UgxNaXAx1pl7S2dWGiN4AaABAg,Nobody: <br>Coby: I’ve never had a hotdog at 8...,Nobody: \nCoby: I’ve never had a hotdog at 8am...,0,2021-08-24T16:07:45Z,2021-08-24T16:07:45Z,0
1,Ugx0awVaxhL-rBNKrNx4AaABAg,I love how Cory is wearing yellow and he hates...,I love how Cory is wearing yellow and he hates...,26,2021-08-23T22:33:57Z,2021-08-23T22:33:57Z,0
2,UgxFSWHcREJ0oP_SR8p4AaABAg,"Boi ur the channel that make my day better, th...","Boi ur the channel that make my day better, th...",0,2021-08-26T01:00:03Z,2021-08-26T01:00:03Z,0
3,UgzpQ5tGyMgiVxxRbGx4AaABAg,Who realized that they used Ty&#39;s OT 28 Coo...,Who realized that they used Ty's OT 28 Cool No...,0,2021-08-25T23:24:07Z,2021-08-25T23:24:07Z,0
4,UgynfhIedxMBipSuj1p4AaABAg,Why is mech arena exactly like war robots but ...,Why is mech arena exactly like war robots but ...,0,2021-08-28T03:00:46Z,2021-08-28T03:00:46Z,0


In [21]:
#get the length value of each comment string

string_length = []
for x in df_comment_stats['comment_text']:
    string_length.append(len(x))

df_comment_stats['text_length'] = string_length

display(df_comment_stats)

,id,comment_display,comment_text,like_count,comment_publish_date,comment_up_date,reply_count,text_length
0,UgxNaXAx1pl7S2dWGiN4AaABAg,Nobody: <br>Coby: I’ve never had a hotdog at 8...,Nobody: \nCoby: I’ve never had a hotdog at 8am...,0,2021-08-24T16:07:45Z,2021-08-24T16:07:45Z,0,52
1,Ugx0awVaxhL-rBNKrNx4AaABAg,I love how Cory is wearing yellow and he hates...,I love how Cory is wearing yellow and he hates...,26,2021-08-23T22:33:57Z,2021-08-23T22:33:57Z,0,55
2,UgxFSWHcREJ0oP_SR8p4AaABAg,"Boi ur the channel that make my day better, th...","Boi ur the channel that make my day better, th...",0,2021-08-26T01:00:03Z,2021-08-26T01:00:03Z,0,77
3,UgzpQ5tGyMgiVxxRbGx4AaABAg,Who realized that they used Ty&#39;s OT 28 Coo...,Who realized that they used Ty's OT 28 Cool No...,0,2021-08-25T23:24:07Z,2021-08-25T23:24:07Z,0,69
4,UgynfhIedxMBipSuj1p4AaABAg,Why is mech arena exactly like war robots but ...,Why is mech arena exactly like war robots but ...,0,2021-08-28T03:00:46Z,2021-08-28T03:00:46Z,0,84
...,...,...,...,...,...,...,...,...
395,UgxVWC9KCTMWdeKAMY54AaABAg,So satisfying to see Tye lose 🤩🤩,So satisfying to see Tye lose 🤩🤩,0,2021-08-24T21:00:27Z,2021-08-24T21:00:27Z,0,32
396,UgybYZpjJSAQkychHQR4AaABAg,We finally don’t see ty in the finale,We finally don’t see ty in the finale,0,2021-08-26T17:20:09Z,2021-08-26T17:20:18Z,0,37
397,UgzaPiOAzKm2n7TKSWV4AaABAg,Can we comment about the fact the some of the ...,Can we comment about the fact the some of the ...,0,2021-08-25T12:11:52Z,2021-08-25T12:11:52Z,0,104
398,UgyBjBhv2yWqwAaTGu14AaABAg,I&#39;d do anything to have the DP editors edi...,I'd do anything to have the DP editors edit my...,7,2021-08-24T05:08:13Z,2021-08-24T05:08:13Z,0,83


In [22]:
#convert comment publish date into a pandas date time format

df_comment_stats['comment_publish_date'] = pd.to_datetime(df_comment_stats.comment_publish_date)

display(df_comment_stats['comment_publish_date'])

#get time that comment was posted
df_comment_stats['comment_time'] = df_comment_stats.comment_publish_date.dt.time

#get the hour of day (1-24) when comment was posted 
df_comment_stats['comment_hr'] = df_comment_stats.comment_publish_date.dt.hour
display(df_comment_stats)

0     2021-08-24 16:07:45+00:00
1     2021-08-23 22:33:57+00:00
2     2021-08-26 01:00:03+00:00
3     2021-08-25 23:24:07+00:00
4     2021-08-28 03:00:46+00:00
                 ...           
395   2021-08-24 21:00:27+00:00
396   2021-08-26 17:20:09+00:00
397   2021-08-25 12:11:52+00:00
398   2021-08-24 05:08:13+00:00
399   2021-08-26 23:42:17+00:00
Name: comment_publish_date, Length: 400, dtype: datetime64[ns, UTC]

,id,comment_display,comment_text,like_count,comment_publish_date,comment_up_date,reply_count,text_length,comment_time,comment_hr
0,UgxNaXAx1pl7S2dWGiN4AaABAg,Nobody: <br>Coby: I’ve never had a hotdog at 8...,Nobody: \nCoby: I’ve never had a hotdog at 8am...,0,2021-08-24 16:07:45+00:00,2021-08-24T16:07:45Z,0,52,16:07:45,16
1,Ugx0awVaxhL-rBNKrNx4AaABAg,I love how Cory is wearing yellow and he hates...,I love how Cory is wearing yellow and he hates...,26,2021-08-23 22:33:57+00:00,2021-08-23T22:33:57Z,0,55,22:33:57,22
2,UgxFSWHcREJ0oP_SR8p4AaABAg,"Boi ur the channel that make my day better, th...","Boi ur the channel that make my day better, th...",0,2021-08-26 01:00:03+00:00,2021-08-26T01:00:03Z,0,77,01:00:03,1
3,UgzpQ5tGyMgiVxxRbGx4AaABAg,Who realized that they used Ty&#39;s OT 28 Coo...,Who realized that they used Ty's OT 28 Cool No...,0,2021-08-25 23:24:07+00:00,2021-08-25T23:24:07Z,0,69,23:24:07,23
4,UgynfhIedxMBipSuj1p4AaABAg,Why is mech arena exactly like war robots but ...,Why is mech arena exactly like war robots but ...,0,2021-08-28 03:00:46+00:00,2021-08-28T03:00:46Z,0,84,03:00:46,3
...,...,...,...,...,...,...,...,...,...,...
395,UgxVWC9KCTMWdeKAMY54AaABAg,So satisfying to see Tye lose 🤩🤩,So satisfying to see Tye lose 🤩🤩,0,2021-08-24 21:00:27+00:00,2021-08-24T21:00:27Z,0,32,21:00:27,21
396,UgybYZpjJSAQkychHQR4AaABAg,We finally don’t see ty in the finale,We finally don’t see ty in the finale,0,2021-08-26 17:20:09+00:00,2021-08-26T17:20:18Z,0,37,17:20:09,17
397,UgzaPiOAzKm2n7TKSWV4AaABAg,Can we comment about the fact the some of the ...,Can we comment about the fact the some of the ...,0,2021-08-25 12:11:52+00:00,2021-08-25T12:11:52Z,0,104,12:11:52,12
398,UgyBjBhv2yWqwAaTGu14AaABAg,I&#39;d do anything to have the DP editors edi...,I'd do anything to have the DP editors edit my...,7,2021-08-24 05:08:13+00:00,2021-08-24T05:08:13Z,0,83,05:08:13,5


In [32]:
#hr of day 8am-4pm working hours, 4pm-12am night, 12am-8am late night 
#put comment time of day in bins, {8-16:'working hours', 16-24:'night', 00-8:'late night'} 
time_bins = []
for time in df_comment_stats['comment_hr']:
    if 16 <= float(time) < 24:
        time_bins.append('night')
    elif 8 <= float(time) < 16:
        time_bins.append('working hours')
    else:
        time_bins.append('late night')
print(time_bins)        
 
df_comment_stats['time_cat'] = time_bins

display(df_comment_stats)




['night', 'night', 'late night', 'night', 'late night', 'night', 'late night', 'night', 'working hours', 'late night', 'working hours', 'late night', 'working hours', 'night', 'night', 'late night', 'late night', 'night', 'working hours', 'late night', 'night', 'night', 'night', 'night', 'night', 'late night', 'late night', 'night', 'late night', 'late night', 'late night', 'late night', 'night', 'late night', 'late night', 'late night', 'late night', 'working hours', 'night', 'night', 'night', 'late night', 'working hours', 'late night', 'night', 'late night', 'night', 'late night', 'late night', 'late night', 'late night', 'late night', 'late night', 'working hours', 'night', 'working hours', 'night', 'night', 'working hours', 'night', 'night', 'working hours', 'late night', 'late night', 'late night', 'night', 'night', 'working hours', 'working hours', 'working hours', 'night', 'working hours', 'late night', 'late night', 'night', 'late night', 'working hours', 'working hours', 'lat

,id,comment_display,comment_text,like_count,comment_publish_date,comment_up_date,reply_count,text_length,comment_time,comment_hr,time_cat
0,UgxNaXAx1pl7S2dWGiN4AaABAg,Nobody: <br>Coby: I’ve never had a hotdog at 8...,Nobody: \nCoby: I’ve never had a hotdog at 8am...,0,2021-08-24 16:07:45+00:00,2021-08-24T16:07:45Z,0,52,16:07:45,16,night
1,Ugx0awVaxhL-rBNKrNx4AaABAg,I love how Cory is wearing yellow and he hates...,I love how Cory is wearing yellow and he hates...,26,2021-08-23 22:33:57+00:00,2021-08-23T22:33:57Z,0,55,22:33:57,22,night
2,UgxFSWHcREJ0oP_SR8p4AaABAg,"Boi ur the channel that make my day better, th...","Boi ur the channel that make my day better, th...",0,2021-08-26 01:00:03+00:00,2021-08-26T01:00:03Z,0,77,01:00:03,1,late night
3,UgzpQ5tGyMgiVxxRbGx4AaABAg,Who realized that they used Ty&#39;s OT 28 Coo...,Who realized that they used Ty's OT 28 Cool No...,0,2021-08-25 23:24:07+00:00,2021-08-25T23:24:07Z,0,69,23:24:07,23,night
4,UgynfhIedxMBipSuj1p4AaABAg,Why is mech arena exactly like war robots but ...,Why is mech arena exactly like war robots but ...,0,2021-08-28 03:00:46+00:00,2021-08-28T03:00:46Z,0,84,03:00:46,3,late night
...,...,...,...,...,...,...,...,...,...,...,...
395,UgxVWC9KCTMWdeKAMY54AaABAg,So satisfying to see Tye lose 🤩🤩,So satisfying to see Tye lose 🤩🤩,0,2021-08-24 21:00:27+00:00,2021-08-24T21:00:27Z,0,32,21:00:27,21,night
396,UgybYZpjJSAQkychHQR4AaABAg,We finally don’t see ty in the finale,We finally don’t see ty in the finale,0,2021-08-26 17:20:09+00:00,2021-08-26T17:20:18Z,0,37,17:20:09,17,night
397,UgzaPiOAzKm2n7TKSWV4AaABAg,Can we comment about the fact the some of the ...,Can we comment about the fact the some of the ...,0,2021-08-25 12:11:52+00:00,2021-08-25T12:11:52Z,0,104,12:11:52,12,working hours
398,UgyBjBhv2yWqwAaTGu14AaABAg,I&#39;d do anything to have the DP editors edi...,I'd do anything to have the DP editors edit my...,7,2021-08-24 05:08:13+00:00,2021-08-24T05:08:13Z,0,83,05:08:13,5,late night


In [51]:
#Does time of comment post correlate with likes? hr of day; 8-4 working hours, 4-12 night, 12-8 late night
unique_time_cat = df_comment_stats['time_cat'].value_counts()
print(unique_time_cat)

#most of the comments were posted at night (4pm - 12pm)

print('\n')

avg_likes = df_comment_stats.groupby(['time_cat']).mean('like_count').sort_values(by = 'like_count', ascending = False, inplace= False)
display(avg_likes)

#comments posted at night got the most likes on average but comments posted late night had a longer text

night            175
late night       114
working hours    111
Name: time_cat, dtype: int64




,like_count,reply_count,text_length,comment_hr
time_cat,,,,
night,4.840000,0.217143,55.308571,19.948571
late night,4.763158,0.131579,70.157895,2.982456
working hours,0.459459,0.027027,52.981982,12.216216


In [46]:
#Pull the top like count by each time category
top_likes_by_cat = df_comment_stats.groupby(['time_cat']).max('like_count').sort_values(by = 'like_count', ascending = False, inplace= False).head(50)
display(top_likes_by_cat)

#The most liked comment comes from the night category

,like_count,reply_count,text_length,comment_hr
time_cat,,,,
night,102,6,245,23
late night,77,3,373,7
working hours,21,1,213,15


In [58]:
#top 10 most liked comment

#found duplicates in initial exploration
#remove duplicates
no_dups = df_comment_stats.drop_duplicates()
print('The API pulled in',len(no_dups), 'unique comments!')

top_10_comments = no_dups.sort_values(by = 'like_count', ascending = False, inplace=False).head(10)

display(top_10_comments)

The API pulled in 252 unique comments!


,id,comment_display,comment_text,like_count,comment_publish_date,comment_up_date,reply_count,text_length,comment_time,comment_hr,time_cat
166,Ugxxc1Cm-McXqXaxt194AaABAg,garrett: i have an advantage in all the athlet...,garrett: i have an advantage in all the athlet...,102,2021-08-23 22:25:31+00:00,2021-08-23T22:25:54Z,3,111,22:25:31,22,night
25,UgwHiDsHYYBft7b6X314AaABAg,Funny how Cody is dry when he says his score o...,Funny how Cody is dry when he says his score o...,77,2021-08-24 03:41:19+00:00,2021-08-24T03:41:19Z,2,111,03:41:19,3,late night
163,UgwBi0TMgxX44QEGeql4AaABAg,The dudes: Talking about strategy\r<br>Coby: “...,The dudes: Talking about strategy\r\nCoby: “I’...,76,2021-08-23 23:53:05+00:00,2021-08-23T23:53:05Z,3,80,23:53:05,23,night
52,UgzRRK-zNZBfItmRT694AaABAg,I watch yalls videos on break in the middle of...,I watch yalls videos on break in the middle of...,56,2021-08-24 06:09:49+00:00,2021-08-24T06:09:49Z,1,136,06:09:49,6,late night
17,UgzfeiLvQadiqmA0cIJ4AaABAg,They definitely didn’t wait 30 minutes before ...,They definitely didn’t wait 30 minutes before ...,40,2021-08-23 22:09:16+00:00,2021-08-23T22:09:16Z,0,61,22:09:16,22,night
87,Ugw50AUI2g2cE4O-g-J4AaABAg,Fun Fact: Everyone Loves Dude Perfect and they...,Fun Fact: Everyone Loves Dude Perfect and they...,39,2021-08-23 22:04:55+00:00,2021-08-23T22:04:55Z,6,73,22:04:55,22,night
168,UgwHxKI965zbH1nsbB94AaABAg,"I have always been Team Tall Guy, he gets fina...","I have always been Team Tall Guy, he gets fina...",31,2021-08-24 04:23:46+00:00,2021-08-24T04:23:46Z,3,77,04:23:46,4,late night
30,Ugwf03eAW9UvccQyiKB4AaABAg,TY: i swim like dolphins<br>Cody: people call ...,TY: i swim like dolphins\nCody: people call me...,30,2021-08-24 03:45:31+00:00,2021-08-24T03:46:08Z,0,213,03:45:31,3,late night
35,Ugz8w4swaMfhk3HYTEh4AaABAg,Everyone concerned about swimming and breath a...,Everyone concerned about swimming and breath a...,30,2021-08-24 02:30:54+00:00,2021-08-24T02:30:54Z,2,111,02:30:54,2,late night
170,UgzPDv9SSacIbpYbmI94AaABAg,I like how they live they’re best life and mak...,I like how they live they’re best life and mak...,27,2021-08-23 22:04:07+00:00,2021-08-23T22:04:07Z,4,63,22:04:07,22,night


In [52]:
#rerun with no duplicates to compare analysis
#Does time of comment post correlate with likes? hr of day; 8-4 working hours, 4-12 night, 12-8 late night
dup_unique_time_cat = no_dups['time_cat'].value_counts()
print(dup_unique_time_cat)

#most of the comments were still posted at night (4pm - 12pm) from our no duplication DF

print('\n')

dup_avg_likes = no_dups.groupby(['time_cat']).mean('like_count').sort_values(by = 'like_count', ascending = False, inplace= False)
display(dup_avg_likes)

#The new analysis of removing duplications showed, late night comments had the highest likes on average 

night            103
late night        78
working hours     71
Name: time_cat, dtype: int64




,like_count,reply_count,text_length,comment_hr
time_cat,,,,
late night,4.602564,0.115385,70.217949,2.948718
night,3.854369,0.174757,57.932039,19.893204
working hours,0.577465,0.042254,53.591549,12.140845


In [54]:
#Does the length of comments have an impact on likes?

#display the top 25 longest comments
display(df_comment_stats.sort_values(by = 'text_length', ascending = False, inplace= False).head(25))

print('\n')

#now display the top 25 longest comments with no duplications
display(no_dups.sort_values(by = 'text_length', ascending = False, inplace= False).head(25))

#This showed that many of the longest comments did not have much likes. Most of the top 25 longest comments had 0 likes.
#The most likes out of this subset was 56 which is the 4th highest likes comment out our entire DF.
#There isn't enough evidence to say that the length of comments have an impact on getting likes. 

print('\n')

#correlation analysis between comment likes and comment length
like_length = df_comment_stats[['like_count', 'text_length']]
correlation = like_length.corr(method='pearson')

display(correlation)

print('\n')

#now do correlation analysis without duplications
dup_like_length = no_dups[['like_count', 'text_length']]
dup_correlation = dup_like_length.corr(method='pearson')


display(dup_correlation)

#Both correlation analysis from our normal DF and no duplication DF between like count and text length showed similar results with a magnitude of around 0.2
#This means that there isn't is a strong correlation between like count and text length 

,id,comment_display,comment_text,like_count,comment_publish_date,comment_up_date,reply_count,text_length,comment_time,comment_hr,time_cat
36,UgyQGPra0UoaWvayjvx4AaABAg,Follow-up to A Glimpse Inside&#39;s comment: ...,Follow-up to A Glimpse Inside's comment: Has ...,0,2021-08-26 01:07:30+00:00,2021-08-26T01:07:30Z,0,373,01:07:30,1,late night
126,UgzWv7ooas9WW4ZdC5R4AaABAg,Jesus is about to return. The rapture is about...,Jesus is about to return. The rapture is about...,0,2021-08-25 16:49:46+00:00,2021-08-25T16:49:46Z,0,245,16:49:46,16,night
326,UgzWv7ooas9WW4ZdC5R4AaABAg,Jesus is about to return. The rapture is about...,Jesus is about to return. The rapture is about...,0,2021-08-25 16:49:46+00:00,2021-08-25T16:49:46Z,0,245,16:49:46,16,night
30,Ugwf03eAW9UvccQyiKB4AaABAg,TY: i swim like dolphins<br>Cody: people call ...,TY: i swim like dolphins\nCody: people call me...,30,2021-08-24 03:45:31+00:00,2021-08-24T03:46:08Z,0,213,03:45:31,3,late night
236,UgwvfD4r2gxrPXK_yJx4AaABAg,Dude Perfect: sponsers Mech Arena <br>Me: <b>I...,Dude Perfect: sponsers Mech Arena \nMe: *I am ...,0,2021-08-27 14:14:48+00:00,2021-08-27T14:14:48Z,0,213,14:14:48,14,working hours
237,UgwidoVQsZ4NdtjXbZt4AaABAg,Dude perfect after OT 29 will you please make ...,Dude perfect after OT 29 will you please make ...,0,2021-08-25 16:37:51+00:00,2021-08-25T16:37:51Z,0,195,16:37:51,16,night
286,UgxzZFBVbwz-AmRHx8J4AaABAg,Dude perfect: Hey we need to rent out your wat...,Dude perfect: Hey we need to rent out your wat...,8,2021-08-24 03:03:21+00:00,2021-08-24T03:03:21Z,0,182,03:03:21,3,late night
190,UgxzZFBVbwz-AmRHx8J4AaABAg,Dude perfect: Hey we need to rent out your wat...,Dude perfect: Hey we need to rent out your wat...,8,2021-08-24 03:03:21+00:00,2021-08-24T03:03:21Z,0,182,03:03:21,3,late night
390,UgxzZFBVbwz-AmRHx8J4AaABAg,Dude perfect: Hey we need to rent out your wat...,Dude perfect: Hey we need to rent out your wat...,8,2021-08-24 03:03:21+00:00,2021-08-24T03:03:21Z,0,182,03:03:21,3,late night
117,UgyD59rc7xFvn5NGuxl4AaABAg,I saw in a video where you guys said that you ...,I saw in a video where you guys said that you ...,0,2021-08-26 08:53:53+00:00,2021-08-26T08:53:53Z,0,179,08:53:53,8,working hours


,id,comment_display,comment_text,like_count,comment_publish_date,comment_up_date,reply_count,text_length,comment_time,comment_hr,time_cat
36,UgyQGPra0UoaWvayjvx4AaABAg,Follow-up to A Glimpse Inside&#39;s comment: ...,Follow-up to A Glimpse Inside's comment: Has ...,0,2021-08-26 01:07:30+00:00,2021-08-26T01:07:30Z,0,373,01:07:30,1,late night
126,UgzWv7ooas9WW4ZdC5R4AaABAg,Jesus is about to return. The rapture is about...,Jesus is about to return. The rapture is about...,0,2021-08-25 16:49:46+00:00,2021-08-25T16:49:46Z,0,245,16:49:46,16,night
30,Ugwf03eAW9UvccQyiKB4AaABAg,TY: i swim like dolphins<br>Cody: people call ...,TY: i swim like dolphins\nCody: people call me...,30,2021-08-24 03:45:31+00:00,2021-08-24T03:46:08Z,0,213,03:45:31,3,late night
236,UgwvfD4r2gxrPXK_yJx4AaABAg,Dude Perfect: sponsers Mech Arena <br>Me: <b>I...,Dude Perfect: sponsers Mech Arena \nMe: *I am ...,0,2021-08-27 14:14:48+00:00,2021-08-27T14:14:48Z,0,213,14:14:48,14,working hours
237,UgwidoVQsZ4NdtjXbZt4AaABAg,Dude perfect after OT 29 will you please make ...,Dude perfect after OT 29 will you please make ...,0,2021-08-25 16:37:51+00:00,2021-08-25T16:37:51Z,0,195,16:37:51,16,night
190,UgxzZFBVbwz-AmRHx8J4AaABAg,Dude perfect: Hey we need to rent out your wat...,Dude perfect: Hey we need to rent out your wat...,8,2021-08-24 03:03:21+00:00,2021-08-24T03:03:21Z,0,182,03:03:21,3,late night
117,UgyD59rc7xFvn5NGuxl4AaABAg,I saw in a video where you guys said that you ...,I saw in a video where you guys said that you ...,0,2021-08-26 08:53:53+00:00,2021-08-26T08:53:53Z,0,179,08:53:53,8,working hours
24,UgzJPUwCdSF35EHItv94AaABAg,I just love it when the notification pops up “...,I just love it when the notification pops up “...,1,2021-08-24 16:27:54+00:00,2021-08-24T16:27:54Z,0,176,16:27:54,16,night
64,Ugy7P2MqZn3VPVd_fG54AaABAg,is nobody gonna talk about how in the last OT ...,is nobody gonna talk about how in the last OT ...,0,2021-08-25 01:34:41+00:00,2021-08-25T01:34:41Z,0,171,01:34:41,1,late night
109,UgwCkDDR08KjZ46Tmzt4AaABAg,Is there a record. for furthest distance a bal...,Is there a record. for furthest distance a bal...,0,2021-08-28 00:02:15+00:00,2021-08-28T00:02:15Z,0,170,00:02:15,0,late night


,like_count,text_length
like_count,1.000000,0.204554
text_length,0.204554,1.000000


,like_count,text_length
like_count,1.000000,0.190686
text_length,0.190686,1.000000


In [41]:
len(df_comment_stats)

400

In [45]:
import csv
df_comment_stats.to_csv('/Users/otan/file/Final_comment_stats.csv', index = False)